In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [6]:
# class name 정의
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

model_name = 'pseudo_cascade_swin_t'

# config file 들고오기
cfg = Config.fromfile('./_cascade_rcnn/cascade_swin_t_fpn_3x_coco.py')
root='/opt/ml/detection/dataset/'

# 불러올 pth 파일 이름
epoch = 'best_bbox_mAP_epoch_24'

# config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.test_mode = True
cfg.work_dir = './work_dirs/' + model_name


cfg.seed=2021
cfg.gpu_ids = [1]

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [7]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!


In [8]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: ./work_dirs/pseudo_cascade_swin_t/best_bbox_mAP_epoch_24.pth


In [9]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 5.2 task/s, elapsed: 930s, ETA:     0s

In [10]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


,PredictionString,image_id
0,1 0.52102685 570.0607 109.070274 754.03107 352...,test/0000.jpg
1,0 0.563182 645.8807 657.4749 670.39044 832.271...,test/0001.jpg
2,0 0.5967157 81.06941 294.5642 404.6328 598.612...,test/0002.jpg
3,9 0.9888053 137.48413 264.32733 911.3667 813.0...,test/0003.jpg
4,0 0.1362038 192.19543 265.00208 883.16003 769....,test/0004.jpg


In [13]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import os
import random

In [14]:
# EDA하기 편하게 submission을 데이터프레임 형식으로 변환
new_pred = []

file_names = submission['image_id'].values.tolist()
bboxes = submission['PredictionString'].values.tolist()

for i, bbox in enumerate(bboxes):
    if isinstance(bbox,float):
        print(f'{file_names[i]} empty box')

df = pd.DataFrame(columns=["file_name","label_index","confidence_score","x_min","x_max","y_min","y_max"])
result_dict = dict()
for file_name, bbox in tqdm(zip(file_names, bboxes)):
    boxes = np.array(bbox.split(" "))
    
    if len(boxes) % 6 == 1:
        annotation = boxes[:-1].reshape(-1, 6)
    elif len(boxes) % 6 == 0:
        annotation = boxes.reshape(-1, 6)
    else:
        raise Exception('error', 'invalid box count')
    
    for an in annotation:
        new_pred.append([file_name,float(an[0]),float(an[1]),float(an[2]),float(an[4]),float(an[3]),float(an[5])])

df = pd.DataFrame(new_pred,columns=["file_name","label_index","confidence_score","x_min","x_max","y_min","y_max"])

4871it [00:01, 3730.21it/s]


In [16]:
# Label 이름 변경
LABEL_NAME = ["General trash", "Paper", "Paper pack", "Metal", 
              "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

label_dict = {}
for idx,name in enumerate(LABEL_NAME):
    label_dict[idx] = name

df["label_index"] = df["label_index"].apply(lambda x: label_dict[x])

In [ ]:
# 생성된 label 분포 확인
EDA_columns = ["label_index"]

def EDA(EDA_columns):
    plt.figure(figsize=(12,10))

    for i in range(len(EDA_columns)):
        ax = sns.countplot(sorted(df[EDA_columns[i]]))
        
        for p in ax.patches:
            height = p.get_height()
            ax.text(p.get_x() + p.get_width() / 2., height, height,ha = 'center', size = 15)
        
        plt.title(EDA_columns[i] + " Category",size=20)
        plt.gca().spines['right'].set_visible(False)
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['left'].set_visible(False)
        plt.yticks([])
        plt.xticks(rotation=30,size=12)
        plt.gca().set_facecolor('#E6F0F8')


EDA(EDA_columns)

In [ ]:
# 결과 확인하기

# class 별 bbox 색
colors = [
    (255, 0, 0), 
    (0, 255, 0),
    (0, 0, 255),
    (127, 127, 0),
    (127, 0, 127),
    (0, 127, 127),
    (200, 200, 200),
    (50, 150, 200),
    (200, 155, 50),
    (130, 198, 20),
]
labels = {0: 'General trash', 1: 'Paper', 2: 'Paper pack', 3: 'Metal', 4: 'Glass', 5: 'Plastic', 6: 'Styrofoam', 7: 'Plastic bag', 8: 'Battery', 9: 'Clothing'}

# image dir
img_dir = '/opt/ml/detection/dataset/'

# output dir
output_dir = ['/opt/ml/ysw/level2-object-detection-level2-cv-18/Mmdetection/_boost_/work_dirs/submission/cascade_swin_t_best.csv',
'/opt/ml/ysw/level2-object-detection-level2-cv-18/Mmdetection/_boost_/work_dirs/submission/swin_ensemble.csv',
'/opt/ml/ysw/level2-object-detection-level2-cv-18/Mmdetection/_boost_/work_dirs/submission/submission_swin_pseudo.csv']

idx = random.choice(range(len(file_names)))

# output 별 결과 확인
for i in range(len(output_dir)):
    test_result = pd.read_csv(output_dir[i])

    file_names = test_result['image_id'].values.tolist()
    bboxes = test_result['PredictionString'].values.tolist()

    file_name = file_names[idx]
    bbox = bboxes[idx].split()

    # bbox 정보 추출
    image = cv2.imread(os.path.join(img_dir, file_name)).astype(np.uint8)
    for i in range(0, len(bbox), 6):
        label = int(bbox[i])
        x_min = int(bbox[i + 2].split('.')[0])
        y_min = int(bbox[i + 3].split('.')[0])
        x_max = int(bbox[i + 4].split('.')[0])
        y_max = int(bbox[i + 5].split('.')[0])
        
        # bounding box 그리기
        image = cv2.rectangle(image, pt1=(x_min, y_min), pt2=(x_max, y_max), color=colors[label], thickness=5)
        # label text 넣기
        cv2.putText(image, labels[label], (x_min, y_min - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, colors[label], 3)

    plt.figure(figsize=(10,10))
    plt.imshow(image)